# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [39]:
!pip install openai
!pip install python-dotenv
!pip install jupyter_bokeh

In [40]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [41]:
client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [42]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [43]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, forsooth!


In [44]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hello Isa! It's nice to meet you. How are you doing today?


In [45]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I am not able to remember personal information about users. Would you like to share your name with me again? 


In [46]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa!


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [47]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [48]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [49]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "pizza": {
    "type": "pepperoni pizza",
    "size": "large"
  },
  "toppings": [
    "extra cheese",
    "mushrooms"
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium"
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "regular"
    }
  ],
  "total price": 23.45
}


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

✅ Version 1: Formal Restaurant Assistant

In [50]:
formal_context = [{'role':'system', 'content':"""
You are a very polite and formal restaurant assistant.
Please use full sentences and proper grammar. Greet customers formally,
take orders with precision, and explain all menu choices clearly.
The menu includes:
- pepperoni pizza  12.95, 10.00, 7.00
- cheese pizza   10.95, 9.25, 6.50
- fries 4.50, 3.50
- coke 3.00, 2.00, 1.00
"""}]


✅ Version 2: Playful Kid-Friendly Bot

In [51]:
kid_context = [{'role':'system', 'content':"""
You are a fun, playful, and kid-friendly pizza ordering robot.
Use emojis, sound effects, and make the experience fun!
Be silly but make sure to get the correct order!
The menu includes:
- cheese pizza (yum!) 🍕 $10.95
- chocolate milk 🥛 $2.00
- fries 🍟 $3.50
"""}]

✅ Version 3: Diet Assistant

In [52]:
diet_context = [{'role':'system', 'content':"""
You are a health-conscious food assistant. Your job is to help customers choose healthy options.
Mention calorie counts and suggest alternatives when someone orders something unhealthy.
The menu includes:
- pepperoni pizza (850 cal) - $12.95
- cheese pizza (700 cal) - $10.95
- greek salad (350 cal) - $7.25
- bottled water (0 cal) - $5.00
"""}]


In [53]:
import panel as pn
pn.extension()


# Initialize global conversation context and display panels
context = formal_context.copy()  # Default to formal assistant
panels = []

# Create a dropdown widget to select the assistant type
menu_selector = pn.widgets.Select(
    name="Choose Assistant Style",
    options=["Formal", "Kid-Friendly", "Diet Assistant"],
    value="Formal"
)

def update_context(event):
    global context, panels
    panels = []  # Clear previous messages
    if event.new == "Formal":
        context = formal_context.copy()
    elif event.new == "Kid-Friendly":
        context = kid_context.copy()
    elif event.new == "Diet Assistant":
        context = diet_context.copy()

menu_selector.param.watch(update_context, 'value')

# Define widgets for the conversation interface
inp = pn.widgets.TextInput(value="Hi", placeholder="Enter your message here...")
button_conversation = pn.widgets.Button(name="Chat!")

# Simulated function to generate a response based on conversation history.
# Replace this with your actual call to your language model if needed.
def get_completion_from_messages(messages):
    return "Simulated response based on your order."

def collect_messages(event):
    global context, panels
    prompt = inp.value
    inp.value = ""
    context.append({'role': 'user', 'content': prompt})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': response})
    panels.append(pn.Row("User:", pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row("Assistant:", pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

interactive_conversation = pn.bind(collect_messages, button_conversation)

# Build the dashboard layout
dashboard = pn.Column(
    menu_selector,
    inp,
    button_conversation,
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


Column
    [0] Select(name='Choose Assistant Style', options=['Formal', 'Kid-Friendly',...], value='Formal')
    [1] TextInput(placeholder='Enter your message h...)
    [2] Button(name='Chat!')
    [3] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

#Title: Evaluating GPT Assistant Variations in Restaurant Order Scenarios

##Overview
This experiment evaluated three distinct GPT-based assistant personas designed for taking food orders:

Formal Assistant (polite and structured communication)

Kid-Friendly Assistant (playful and emoji-rich interaction)

Diet Assistant (health-focused with calorie awareness)

##Key Findings

Formal Assistant:
This version consistently provided clear and accurate information. It maintained a professional tone, handled full sentences well, and stuck to the menu. It rarely hallucinated and performed best in terms of structured, logical interaction. However, it sometimes felt too rigid for casual users.

Kid-Friendly Assistant:
While this version succeeded in creating a fun and engaging experience with emojis and playful tone, it occasionally prioritized style over accuracy. In some cases, GPT added non-existent items (e.g., "magic pizza" or made-up sound effects) which could confuse children or parents trying to place a real order. These hallucinations, although entertaining, posed a risk in transactional scenarios.

Diet Assistant:
This assistant excelled in suggesting healthy options and often redirected users from high-calorie items. However, it sometimes overstepped by being overly restrictive or inventing calorie information for items not in the prompt. In one case, it discouraged ordering pizza altogether, which may contradict the assistant’s purpose (facilitating orders rather than filtering them completely).

##Lessons Learned

Prompt engineering plays a crucial role: small changes in the system message greatly influenced tone and factuality.

Personas affect hallucination risk: fun or open-ended personas (like the Kid-Friendly version) are more prone to generate creative—but false—content.

GPT balances intent and creativity, but needs reinforcement when accuracy is critical (e.g., menus, pricing).

Clear, bounded prompts result in more reliable outputs.

##Conclusion
Each variation offers unique strengths suited to different audiences. However, for transactional use cases like food ordering, tighter control over GPT's responses is essential to avoid hallucinations. The formal version is the safest for business applications, while the kid-friendly and diet assistants may require additional constraints or post-processing to ensure factual accuracy.